# Load data

In [3]:
from kubetools import get_all, delete, top, get_gcloud_node_info
from subprocess import check_output
import pandas as pd

# Grab all pods + pod resources information, merge into one df
df_top = top('pod')
df_get = get_all('pod')
df = pd.merge(df_top, df_get, on='name', how='outer')

# Convert age to hours and sort
df['age'] /= 60.0
df = df.sort_values(['age', 'cpu'], ascending=False)

# Determine if this is a user/prod/etc pod
df['kind'] = df['name'].map(lambda a: a.split('-')[0])

df['restarts'] = pd.to_numeric(df['restarts'])

# Grab node information from both kubectl and gcloud
nodes = get_all('node')
nodes_gcloud = get_gcloud_node_info().rename(columns={'external_ip': 'external-ip', 'status': 'status_gcloud'})
nodes = pd.merge(nodes, nodes_gcloud, on=['name', 'external-ip'])
nodes = nodes[['name', 'internal_ip', 'external-ip', 'status', 'roles', 'age', 'version', 'os-image',
               'kernel-version', 'container-runtime', 'zone', 'machine_type', 'status_gcloud']]

# Pod management

## Deleting pods by query

In [ ]:
# Remove pods in error state
# q = 'status in ["Completed", "Error", "Evicted"]'

# Delete non-user pods on a node
node = "gke-prod-a-ssd-pool-32-134a959a-5mpp"
q = 'fullnode == "{}" and kind not in ["jupyter", "build"]'.format(node)

delete_pods = df.query(q)
print(len(delete_pods))
delete_pods

In [6]:
for nm in delete_pods['name'].values:
    delete(nm, force=True)

Deleted pod: build-kmader-2dquantitative-2dbig-2dimaging-2d201-50d2b9-9cf0fc
Deleted pod: build-samuelwiqvist-2dabc-d431fa-047180
Deleted pod: prod-nginx-ingress-controller-7b4fdbdcc8-b2k9v


## Delete a specific pod

In [5]:
nm = 'binder-5b8b4479cb-gzkq8'
delete(nm)

Deleted pod: binder-5b8b4479cb-gzkq8


# Node management

## Cordon node

In [1]:
!kubectl cordon gke-prod-a-ssd-pool-32-134a959a-qvdt

node "gke-prod-a-ssd-pool-32-134a959a-qvdt" cordoned


In [2]:
!kubectl uncordon gke-prod-a-ssd-pool-32-134a959a-5mpp

node "gke-prod-a-ssd-pool-32-134a959a-5mpp" uncordoned


## Drain a node

In [ ]:
! kubectl drain --force --delete-local-data --ignore-daemonsets --grace-period=0 gke-prod-a-ssd-pool-32-134a959a-s2pn

In [ ]:
df.query('node ==')

## Delete node

In [12]:
!kubectl delete node gke-prod-a-ssd-pool-32-134a959a-s2pn

b'node "gke-prod-a-ssd-pool-32-134a959a-tvpp" deleted\n'